In [25]:
# %load_ext autoreload
%autoreload 2
import numpy as np 
import sys
sys.path.append('..')
from capacity_management.src.da import DeferredAcceptance

In [36]:
n = 60 # num students
qs = np.full(3, 22) # true capacity
priority_category_size = n//3
typeA_per_priority = 15 # numbsd of students who prefer 1 > 2 > 3 (type A), remainder prefer 1 > 3 > 2 (type B)
assert typeA_per_priority < n / 3

In [32]:
def generate_priorities(n):
    priorities = np.random.uniform(size=(n, 3))
    for i in range(3):
        priorities[n//3*i:n//3*(i+1),i] += 1
    return priorities

In [33]:
def generate_preferences(n, typeA_per_priority, priority_category_size):
    preferences = np.zeros((n,3))
    for i in range(3):
        preferences[n//3*i:n//3*i + typeA_per_priority,1] = 1
        preferences[n//3*i + typeA_per_priority:n//3*i +priority_category_size,1] = 2

        preferences[n//3*i:n//3*i + typeA_per_priority,2] = 2
        preferences[n//3*i + typeA_per_priority:n//3*i +priority_category_size,2] = 1
    return preferences

def generate_types(n, typeA_per_priority, priority_category_size):
    types = ['' for i in range(n)] 
    for i in range(3):
        types[n//3*i:n//3*i + typeA_per_priority] = [f'A{i}' for j in range(typeA_per_priority)]
        types[n//3*i + typeA_per_priority:n//3*i +priority_category_size] = [f'B{i}' for j in range(priority_category_size-typeA_per_priority)]

    return types

In [34]:
priorities = generate_priorities(n)
preferences = generate_preferences(n, typeA_per_priority, priority_category_size)
types = generate_types(n, typeA_per_priority, priority_category_size)

da = DeferredAcceptance(preferences, priorities, qs)
assignment = da.da()

In [37]:
def count_types_per_school(assignment, types):
    type_count = {}
    for school, students in assignment.items():
        student_types = [types[i] for i in students]
        vals, counts = np.unique(student_types, return_counts=True)
        type_count[school] = dict(zip(vals, counts))
    return type_count
    
count_types_per_school(assignment, types)

{0: {'A0': 15, 'A2': 2, 'B0': 5},
 1: {'A1': 15, 'A2': 7},
 2: {'A2': 6, 'B1': 5, 'B2': 5}}